<center>

<h1 style="font-size: 40px;">MiniMax</h1>

</center>

The goal of the exercise is to implement the minimax algorithm with alpha-beta pruning. For different moves of equal quality, the algorithm should return one of them randomly. Then, you should utilize the implementation to compare the effectiveness of the algorithm for different depths of search for a simple game with the following rules:
- At the beginning of the game, there is a set of N identical tokens, from which players take turns removing tokens.
- Two players take turns making moves.
- During their turn, a player removes from 1 to K tokens.
- The player who takes the last token loses the game.
- Experiments should consider the situation where $K=3$, and N is a number drawn from the range $8-20$ (with a uniform distribution).

In [ ]:
%pip install pandas
%pip install ipython
%pip install ipympl
%pip install tabulate
%pip install ipywidgets
%matplotlib widget

import random
import pandas as pd
from IPython.display import display_html
from timeit import default_timer as timer

# Heuristic function

In [2]:
def heuristic_evaluation(N, move_max):
  '''
  params:
    N: int, number of tokens
    move_max: bool, True if it is max's turn to move, False if it is min's turn to move
  '''
  if N == 0:
    if move_max:
      return 1
    else:
      return -1
  else:
      return 0

# Mini max algorithm

In [3]:
def mini_max(N, depth, move_max, K, alpha, beta, pruning=True):
  '''
  params:
    N: int, number of tokens
    depth: int, depth of the tree
    move_max: bool, True if it is max's turn to move, False if it is min's turn to move
    K: int, max number of tokens that can be removed in a turn
    alpha: int, alpha value for pruning
    beta: int, beta value for pruning
    pruning: bool, True if pruning is enabled, False if pruning is disabled
  '''
  if depth == 0  or N == 0:
    return heuristic_evaluation(N, move_max)
  # Legal moves are from 1 to K or N, whichever is smaller
  legal_moves = [i for i in range(1, min(N, K) + 1)]

  if move_max:
    max_eval = float('-inf')
    for move in legal_moves:
      new_N = N - move
      # recursive call used to evaluate the next level
      eval = mini_max(new_N, depth-1, not move_max, K, alpha, beta, pruning)
      max_eval = max(max_eval, eval)
      # pruning
      if pruning:
        alpha = max(alpha, max_eval)

        if (alpha >= beta):
          break
      
    return max_eval
  
  else:
    min_eval = float('inf')
    for move in legal_moves:
      new_N = N - move
      # recursive call used to evaluate the next level
      eval = mini_max(new_N, depth-1, not move_max, K, alpha, beta, pruning)
      min_eval = min(min_eval, eval)
      # pruning
      if pruning:
        beta = min(beta, min_eval)
        
        if (alpha >= beta):
          break

    return min_eval

# Find the best move in current tree

In [4]:
def best_move(N, depth, move_max, K, pruning=True):
    '''
    params:
      N: int, number of tokens
      depth: int, depth of the tree
      move_max: bool, True if it is max's turn to move, False if it is min's turn to move
      K: int, max number of tokens that can be removed in a turn
      pruning: bool, True if pruning is enabled, False if pruning is disabled
    '''
    # Set alpha and beta to -inf and inf respectively
    alpha = float('-inf')
    beta = float('inf')
    legal_moves = [i for i in range(1, min(N, K) + 1)]
    best_moves = []
    # If there are no legal moves, return the heuristic evaluation
    if not legal_moves:
            return heuristic_evaluation(N, move_max)
    max_eval = -1
    min_eval = 1
    for move in legal_moves:
        # If it is the max player's turn, find the move with the highest evaluation
        if move_max:
            new_N = N - move
            eval = mini_max(new_N, depth, not move_max, K, alpha, beta, pruning)
            if eval > max_eval:
                best_moves = [move]
                max_eval = eval

            elif eval == max_eval:
                best_moves.append(move)
        # If it is the min player's turn, find the move with the lowest evaluation
        else:
            new_N = N - move
            eval = mini_max(new_N, depth,not move_max, K, alpha, beta, pruning)

            if eval < min_eval:
                best_moves = [move]
                min_eval = eval

            elif eval == min_eval:
                best_moves.append(move)

    return random.choice(best_moves)

# Simulate the result of game

In [5]:
def game_simulation(N, depth, move_max, K, pruning=True):
    '''
    params:
      N: int, number of tokens
      depth: int, depth of the tree
      move_max: bool, True if it is max's turn to move, False if it is min's turn to move
      K: int, max number of tokens that can be removed in a turn
      pruning: bool, True if pruning is enabled, False if pruning is disabled
    '''
    tE = timer()
    move_max = True
    while N > 0:
        move = best_move(N, depth, move_max, K, pruning)
        N = N - move
        if move_max:
            if N == 0:
                tQ = timer()
                return False, tQ - tE
        else:
            if N == 0:
                tQ = timer()
                return True, tQ - tE
        move_max = not move_max

# Check the result of $n$ games

In [6]:
def chances_to_win(N, depth, move_max, K, n, pruning=True):
    wins = 0
    time = []
    for i in range(n):
        wins += game_simulation(N, depth, move_max, K, pruning)[0]
        time.append(game_simulation(N, depth, move_max, K, pruning)[1])
        avg_time = sum(time)/len(time)
        sigma = (sum([(i - avg_time)**2 for i in time])/n)**0.5
    return f'{wins*100/n}%', f'{round(avg_time, 4)} ± {round(sigma, 4)}'

# Return the DataFrame using random distribution of tokens initial value

In [7]:
def random_tokens_game_data(depth, move_max, K, repeat, range_random, rows, pruning=True):
    df = pd.DataFrame(columns=['range N', 'K', 'depth', 'chances'])
    for j in range(rows):
        win = 0
        for i in range(repeat):
            N = random.randint(range_random[0], range_random[1])
            win += game_simulation(N, depth, move_max, K, pruning)[0]
        chances = f'{win*100/repeat}%'
        new_df = pd.DataFrame({'range N':[range_random], 'K': [K], 'depth':[depth], 'chances': [chances]})
        df = pd.concat([df, new_df], ignore_index=True)
    
    return df

# Return the DataFrame for each initial tokens value

In [8]:
def tokens_game_data(depth, move_max, K, repeat, range_values, pruning=True):
    df = pd.DataFrame(columns=['N', 'K', 'depth', 'chances'])
    for i in range(range_values[0], range_values[1] + 1):
        chances = chances_to_win(i, depth, move_max, K, repeat, pruning)[0]
        new_df = pd.DataFrame({'N': [i], 'K': [K], 'depth':[depth], 'chances': [chances]})
        df = pd.concat([df, new_df], ignore_index=True)
    
    return df

# Return the DataFrame of elapsed times for each initial tokens value

In [9]:
def elapsed_time(depth, move_max, K, repeat, range_values, pruning=True):
    df = pd.DataFrame(columns=['N', 'K', 'depth', 'avg ± sigma'])
    for i in range(range_values[0], range_values[1] + 1):
        avg_sigma = chances_to_win(i, depth, move_max, K, repeat, pruning)[1]
        new_df = pd.DataFrame({'N': [i], 'K': [K], 'depth':[depth], 'avg ± sigma':[avg_sigma]})
        df = pd.concat([df, new_df], ignore_index=True)

    return df


Tests for varying values of $K$ with a large depth of search:
- $K=3$
- $K=4$
- $K=5$

In [10]:
depth = 10
move_max = True
repeat = 100
range_values = [5, 20]
K = 3
df1 = tokens_game_data(depth, move_max, K, repeat, range_values)

K = 4
df2 = tokens_game_data(depth, move_max, K, repeat, range_values)

K = 5
df3 = tokens_game_data(depth, move_max, K, repeat, range_values)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('K = 3')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('K = 4')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('K = 5')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)


,N,K,depth,chances
0,5,3,10,0.0%
1,6,3,10,100.0%
2,7,3,10,100.0%
3,8,3,10,100.0%
4,9,3,10,0.0%
5,10,3,10,100.0%
6,11,3,10,100.0%
7,12,3,10,100.0%
8,13,3,10,0.0%
9,14,3,10,100.0%


In the presented tables, we notice that whether the maximizing player wins depends on the initial number of tokens. Whenever there are $K+1$ tokens, the algorithm loses in 100% of cases.

Tests for decreased depth of search:

In [11]:
move_max = True
K = 3
values = 100
depth = 3
range_values = [5, 20]
df1 = tokens_game_data(depth, move_max, K, values, range_values)

depth = 5
df2 = tokens_game_data(depth, move_max, K, values, range_values)

depth = 7
df3 = tokens_game_data(depth, move_max, K, values, range_values)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('depth 5')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('depth 6')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('depth 7')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)


,N,K,depth,chances
0,5,3,3,0.0%
1,6,3,3,100.0%
2,7,3,3,100.0%
3,8,3,3,100.0%
4,9,3,3,0.0%
5,10,3,3,36.0%
6,11,3,3,55.0%
7,12,3,3,65.0%
8,13,3,3,42.0%
9,14,3,3,42.0%


It can be observed that the higher we increase the depth value, the greater certainty we have about the outcome of the game.

In [12]:
depth = 10
move_max = True
K = 3
repeat = 10
range_values = [15, 40]
tokens_game_data(depth, move_max, K, repeat, range_values)

,N,K,depth,chances
0,15,3,10,100.0%
1,16,3,10,100.0%
2,17,3,10,0.0%
3,18,3,10,100.0%
4,19,3,10,100.0%
5,20,3,10,100.0%
6,21,3,10,0.0%
7,22,3,10,100.0%
8,23,3,10,40.0%
9,24,3,10,40.0%


It can be observed that even when choosing a high depth such as 10, we are still unable to accurately determine the outcome of the game for larger initial token values.

Tests for random values from the intervals:
- $8-20$
- $21-32$
- $33-45$

using a uniform distribution

In [13]:
depth = 5
move_max = True
repeat = 100
rows = 10
K = 3

range_random = [8, 20]
df1 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

range_random = [21, 32]
df2 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

range_random = [33, 45]
df3 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('range N = [8, 20]')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('range N = [21, 32]')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('range N = [33, 45]')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

,range N,K,depth,chances
0,"[8, 20]",3,5,61.0%
1,"[8, 20]",3,5,57.0%
2,"[8, 20]",3,5,57.0%
3,"[8, 20]",3,5,61.0%
4,"[8, 20]",3,5,49.0%
5,"[8, 20]",3,5,52.0%
6,"[8, 20]",3,5,65.0%
7,"[8, 20]",3,5,58.0%
8,"[8, 20]",3,5,67.0%
9,"[8, 20]",3,5,56.0%


It can be observed that at the specified depth of 5, the chances of winning do not change regardless of the range of values we choose.

In [14]:
depth = 10
move_max = True
repeat = 100
rows = 10
K = 3

range_random = [8, 20]
df1 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

range_random = [21, 32]
df2 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

range_random = [33, 45]
df3 = random_tokens_game_data(depth, move_max, K, repeat, range_random, rows)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('range N = [8, 20]')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('range N = [21, 32]')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('range N = [33, 45]')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

,range N,K,depth,chances
0,"[8, 20]",3,10,77.0%
1,"[8, 20]",3,10,74.0%
2,"[8, 20]",3,10,75.0%
3,"[8, 20]",3,10,78.0%
4,"[8, 20]",3,10,75.0%
5,"[8, 20]",3,10,78.0%
6,"[8, 20]",3,10,74.0%
7,"[8, 20]",3,10,76.0%
8,"[8, 20]",3,10,67.0%
9,"[8, 20]",3,10,81.0%


Increasing the depth only affects the chances of winning for values of N from the range $8-20$. If we wanted to increase the chances of winning for higher ranges, we would need to further increase the depth, but this would require significant computational resources. The execution time would increase dramatically.

So far, we have been using alpha-beta pruning, which has saved us a lot of time. Now, let's check exactly how much.

In [15]:
depth = 5
move_max = True
K = 3
range_values = [8, 20]
n = 10

df1 = elapsed_time(depth, move_max, K, n, range_values)
df2 = elapsed_time(depth, move_max, K, n, range_values, pruning=False)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('with pruning')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('without pruning')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,N,K,depth,avg ± sigma
0,8,3,5,0.0001 ± 0.0
1,9,3,5,0.0002 ± 0.0
2,10,3,5,0.0004 ± 0.0
3,11,3,5,0.0006 ± 0.0001
4,12,3,5,0.0007 ± 0.0004
5,13,3,5,0.0007 ± 0.0001
6,14,3,5,0.0007 ± 0.0001
7,15,3,5,0.0008 ± 0.0001
8,16,3,5,0.001 ± 0.0002
9,17,3,5,0.001 ± 0.0002


The differences are indeed small for the given range of values of N and depth.

In [16]:
depth = 10
move_max = True
K = 3
range_values = [8, 20]
n = 10

df1 = elapsed_time(depth, move_max, K, n, range_values)
df2 = elapsed_time(depth, move_max, K, n, range_values, pruning=False)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('with pruning')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('without pruning')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,N,K,depth,avg ± sigma
0,8,3,10,0.0002 ± 0.0003
1,9,3,10,0.0003 ± 0.0
2,10,3,10,0.0007 ± 0.0002
3,11,3,10,0.001 ± 0.0003
4,12,3,10,0.0015 ± 0.0007
5,13,3,10,0.0029 ± 0.0008
6,14,3,10,0.0048 ± 0.0005
7,15,3,10,0.0057 ± 0.0006
8,16,3,10,0.01 ± 0.0039
9,17,3,10,0.0136 ± 0.001


Increasing the depth, we can observe that the computation time can increase even up to 5 times.

In [17]:
depth = 10
move_max = True
K = 3
range_values = [20, 30]
n = 10

df1 = elapsed_time(depth, move_max, K, n, range_values)
df2 = elapsed_time(depth, move_max, K, n, range_values, pruning=False)

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('with pruning')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('without pruning')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,N,K,depth,avg ± sigma
0,20,3,10,0.019 ± 0.0024
1,21,3,10,0.0254 ± 0.0017
2,22,3,10,0.0293 ± 0.0012
3,23,3,10,0.031 ± 0.0035
4,24,3,10,0.0297 ± 0.0028
5,25,3,10,0.0316 ± 0.002
6,26,3,10,0.0328 ± 0.0029
7,27,3,10,0.0397 ± 0.0094
8,28,3,10,0.0367 ± 0.0028
9,29,3,10,0.0395 ± 0.003


When we increased the values of $N$ and depth, we can observe that the computation time increases very rapidly. With alpha-beta pruning, for $N=30$ and depth $10$, the computation time is shortened by a factor of $20$.

# Conclusions:
- For the given maximum token removal value $K$ and a sufficiently large depth, we can determine the victory of the maximizing player with 100% certainty. Specifically, every $K+1$ tokens result in a loss.
- As the depth of search increases, the chances of winning increase, but so does the computation time.
- If we want to be certain about our result, we need to adjust our depth to be at most a few values lower than the initial token value $N$.
- For large initial values of N and depth, alpha-beta pruning is capable of significantly reducing computation time.

# Mini-game demonstrating the algorithm's operation

In [20]:
%pip install pygame

import pygame

# Choose the number of tokens to start with
INITIAL_TOKENS = 20
DEPTH = 15


WIDTH, HEIGHT = 300, 200
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
FPS = 60
SQUARE_SIZE = 32

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Token Game")
clock = pygame.time.Clock()

pygame.init()
pygame.font.init()

number_font = pygame.font.SysFont(None, 16)
token_font = pygame.font.SysFont(None, 24)

image = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE))
image.fill(WHITE)

button1 = pygame.Rect((40, 150, SQUARE_SIZE, SQUARE_SIZE))
num1 = number_font.render("1", True, BLUE)

button2 = pygame.Rect((140, 150, SQUARE_SIZE, SQUARE_SIZE))
num2 = number_font.render("2", True, BLUE)

button3 = pygame.Rect((240, 150, SQUARE_SIZE, SQUARE_SIZE))
num3 = number_font.render("3", True, BLUE)

reset_button = pygame.Rect((260, 100, SQUARE_SIZE, SQUARE_SIZE))
reset_text = number_font.render("reset", True, BLUE)

def computer_move(TOKENS):
    MOVE = best_move(TOKENS, DEPTH, True, 3)
    return MOVE

def print_text(text, x, y, color=WHITE):
    text = token_font.render(text, True, color)
    screen.blit(text, (x, y))

def draw_button(button, num):
    screen.blit(image, button)
    screen.blit(num, (button.x + SQUARE_SIZE // 2 - num.get_width() // 2, button.y + SQUARE_SIZE // 2 - num.get_height() // 2))

TAKEN = 0
MOVE = 0
lost = False
win = False
reset_pressed = False

TOKENS = INITIAL_TOKENS

def handle_player_input(event):
    global TAKEN, reset_pressed
    moved = False

    if event.type == pygame.MOUSEBUTTONDOWN:
        if button1.collidepoint(event.pos):
            TAKEN = 1
            moved = True
        elif button2.collidepoint(event.pos):
            TAKEN = 2
            moved = True
        elif button3.collidepoint(event.pos):
            TAKEN = 3
            moved = True
        elif reset_button.collidepoint(event.pos):
            reset_pressed = True
    return moved

def handle_reset():
    global TAKEN, MOVE, TOKENS, lost, win
    TAKEN = 0
    MOVE = 0
    TOKENS = INITIAL_TOKENS
    lost = False
    win = False

running = True

while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        moved = handle_player_input(event)

    screen.fill(BLACK)

    if reset_pressed:
        handle_reset()
        reset_pressed = False

    if moved:
        TOKENS -= TAKEN
        if TOKENS <= 0:
            TOKENS = 0
            lost = True

        MOVE = computer_move(TOKENS)
        TOKENS -= MOVE
        if TOKENS <= 0:
            TOKENS = 0
            win = True

    moved = False

    draw_button(button1, num1)
    draw_button(button2, num2)
    draw_button(button3, num3)
    draw_button(reset_button, reset_text)

    if TAKEN != 0 and TOKENS > 0:
        print_text("You took: " + str(TAKEN), 10, 30)

    if MOVE != 0 and TOKENS > 0:
        print_text("Computer took: " + str(MOVE), 10, 50)

    print_text("Tokens: " + str(TOKENS), 10, 10)

    if lost:
        print_text("You lost!", 10, 70, RED)
        print_text("Click 'reset' to play again.", 10, 90)

    elif win:
        print_text("You won!", 10, 70, GREEN)
        print_text("Click 'reset' to play again.", 10, 90)

    pygame.display.update()

pygame.quit()

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
